In [1]:
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/Github/

/content/drive/MyDrive/Github


In [ ]:
!git init

In [8]:
!git pull https://github.com/rohit-khoiwal-30/msgmt.git

From https://github.com/rohit-khoiwal-30/msgmt
 * branch            HEAD       -> FETCH_HEAD
Updating a77fca4..01b7e22
Fast-forward
 augment_train.csv | 8205 +++++++++++++++++++++++++++++++++++++++++++++++++++++
 preprocess.ipynb  | 2176 +++++++-------
 2 files changed, 9392 insertions(+), 989 deletions(-)
 create mode 100644 augment_train.csv


In [9]:
df = pd.read_csv("augment_train.csv")
df_test = pd.read_csv("evaluation.csv")

In [10]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Input
from keras.regularizers import l2

try:
    import tensorflow_hub as hub
except ModuleNotFoundError:
    %pip install -q tensorflow_hub
    import tensorflow_hub as hub
    
try:
    from keras_self_attention import SeqSelfAttention
except ModuleNotFoundError:
    %pip install -q keras-self-attention
    from keras_self_attention import SeqSelfAttention

import numpy as np


# %pip install -q transformers
# %pip install -q -U tensorflow-text
# %pip install -q tf-models-official==2.7.0

import math as m

In [11]:
huburl = "https://tfhub.dev/google/universal-sentence-encoder/4" 
fine_tuned_module_object = hub.load(huburl)

In [12]:
def gen_random_batch(in_groups, batch_halfsize = 8):
    text_batch, reason_batch, y_hat = [], [], []
    all_groups = list(range(in_groups[0].shape[0]))
    for match_group in [True, False]:
        group_idx = np.random.choice(all_groups, size = batch_halfsize)
        text_batch += [in_groups[0][c_idx] for c_idx in group_idx]
        if match_group:
            b_group_idx = group_idx
            y_hat += [1]*batch_halfsize
        else:
            # anything but the same group
            non_group_idx = [np.random.choice([i for i in all_groups if i!=c_idx]) for c_idx in group_idx] 
            b_group_idx = non_group_idx
            y_hat += [0]*batch_halfsize
            
        reason_batch += [in_groups[1][c_idx] for c_idx in b_group_idx]
            
    return np.stack(text_batch, 0), np.stack(reason_batch, 0), np.stack(y_hat, 0)

def siam_gen(in_groups, batch_size = 32):
    while True:
        text_stack, reason_stack, y_hatstack = gen_random_batch(in_groups, batch_size//2)
        yield [text_stack, reason_stack], y_hatstack

def euclidean_distance(vectors):
    (featsA, featsB) = vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

def contrastive_loss(y, preds, margin=1):
    # explicitly cast the true class label data type to the predicted
    # class label data type (otherwise we run the risk of having two
    # separate data types, causing TensorFlow to error out)
    y = tf.cast(y, preds.dtype)
    # calculate the contrastive loss between the true labels and
    # the predicted labels
    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)
    return loss

In [13]:
def accuracy(y_true, y_pred):
    results = (y_pred <= 0.5).astype(int).squeeze()
    return np.mean([y_true == results])

In [14]:
def get_model():
    shared_embedding_layer1 = hub.KerasLayer(fine_tuned_module_object, trainable=True, name="first")
    shared_embedding_layer2 = hub.KerasLayer(fine_tuned_module_object, trainable=True, name="second")

    left_input = tf.keras.Input(shape=(), dtype=tf.string)
    right_input = tf.keras.Input(shape=(), dtype=tf.string)

    embedding_left_output= shared_embedding_layer1(left_input)
    a = tfl.Dropout(0.3)(embedding_left_output)
    a = tfl.Dense(64, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    a = tfl.Dropout(rate=0.2)(a)
    a = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(a)
    # a = tfl.BatchNormalization()(a)
    # a = tfl.Activation('relu')(a)
    a = tfl.Lambda(lambda x: K.l2_normalize(x,axis=-1))(a)

    embedding_right_output= shared_embedding_layer2(right_input)
    b = tfl.Dropout(0.3)(embedding_right_output)
    b = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(b)
    # b = tfl.BatchNormalization()(b)
    # b = tfl.Activation('relu')(b)
    b = tfl.Lambda(lambda x: K.l2_normalize(x,axis=-1))(b)

    distance = tfl.Lambda(euclidean_distance)([a, b])
    model = Model([left_input, right_input], distance)
    return model

In [26]:
model = get_model()

In [27]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 first (KerasLayer)             (None, 512)          256797824   ['input_3[0][0]']                
                                                                                                  
 dropout_3 (Dropout)            (None, 512)          0           ['first[0][0]']                  
                                                                                                  
 dense_3 (Dense)                (None, 64)           32832       ['dropout_3[0][0]']              
                                                                                            

In [28]:
model.compile(optimizer="adam", loss = contrastive_loss, metrics = [tf.metrics.BinaryAccuracy()])

In [29]:
loss_history = model.fit(siam_gen([df["texts"], df["reason"]], 64), 
                         validation_data = siam_gen([df_test["text"], df_test["reason"]], 64),
                         steps_per_epoch=50, validation_steps=10, epochs = 30, 
                         verbose = True, use_multiprocessing=True)

Epoch 1/30
50/50 [==============================] - 17s 211ms/step - loss: 0.4707 - binary_accuracy: 0.3544 - val_loss: 0.4186 - val_binary_accuracy: 0.3203
Epoch 2/30
50/50 [==============================] - 11s 221ms/step - loss: 0.3101 - binary_accuracy: 0.1056 - val_loss: 0.3937 - val_binary_accuracy: 0.3094
Epoch 3/30
50/50 [==============================] - 10s 202ms/step - loss: 0.2649 - binary_accuracy: 0.0697 - val_loss: 0.3770 - val_binary_accuracy: 0.3469
Epoch 4/30
50/50 [==============================] - 10s 192ms/step - loss: 0.2320 - binary_accuracy: 0.0722 - val_loss: 0.3534 - val_binary_accuracy: 0.3266
Epoch 5/30
50/50 [==============================] - 10s 194ms/step - loss: 0.2033 - binary_accuracy: 0.0669 - val_loss: 0.3334 - val_binary_accuracy: 0.3281
Epoch 6/30
50/50 [==============================] - 10s 194ms/step - loss: 0.1754 - binary_accuracy: 0.0594 - val_loss: 0.3268 - val_binary_accuracy: 0.3344
Epoch 7/30
50/50 [==============================] - 10s 19

In [32]:
y_preds = model.predict([df["texts"], df["reason"]])
accuracy(df["label"],y_preds)

0.9809732894255397

In [33]:
y_preds = model.predict([df_test["text"], df_test["reason"]])
accuracy(df_test["label"],y_preds)

0.5971111111111111

In [ ]:
model.save("models/useEUCL/")